In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np
import requests
from io import BytesIO

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

# 데이터셋 로드 및 분할
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# 데이터 전처리 함수
def preprocess(image, label):
    image = tf.image.resize(image, (150, 150))  # 입력 크기 조정
    image = image / 255.0  # 정규화
    return image, label

# 데이터 파이프라인 구성
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

ds_train = ds_train.map(preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_val = ds_val.map(preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_test = ds_test.map(preprocess, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.AMYQRD_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [3]:
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

# Xception 모델 구성
base_model = Xception(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # 사전 학습된 가중치를 고정

# 사용자 정의 분류기 추가
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),  # 과적합 방지
    layers.Dense(1, activation='sigmoid')  # 이진 분류
])

# 모델 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 모델 저장 경로
model_save_path = './xception_cats_vs_dogs_model.keras'

# 모델 학습
history = model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_val
)

# 모델 저장
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

# 테스트 데이터 평가
test_loss, test_accuracy = model.evaluate(ds_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 63s 85ms/step - accuracy: 0.9448 - loss: 0.1357 - val_accuracy: 0.9695 - val_loss: 0.0802
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 62s 66ms/step - accuracy: 0.9696 - loss: 0.0780 - val_accuracy: 0.9712 - val_loss: 0.0803
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 38s 64ms/step - accuracy: 0.9749 - loss: 0.0660 - val_accuracy: 0.9729 - val_loss: 0.0809
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 39s 67ms/step - accuracy: 0.9777 - loss: 0.0597 - val_accuracy: 0.9725 - val_loss: 0.0805
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 40s 68ms/step - accuracy: 0.9803 - loss: 0.0532 - val_accuracy: 0.9716 - val_loss: 0.0831
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 68ms/step - accuracy: 0.9818 - loss: 0.0495 - val_accuracy: 0.9733 - val_loss: 0.0888
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 70ms/step - accuracy: 0.9831 - loss: 0.0428 - val_accuracy: 0.9703 - val_loss: 0.0944
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━